# CIFAR-10 data set
## Here are the classes in the dataset, as well as 10 random images from each:
<img src="cifar-10-classes.jpg" class="cifar-sample" width = 600>

In [309]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.la
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

In [310]:
train_path_entry = ["/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_1",
"/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_2",
"/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_3",
"/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_4",
"/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_5"]
path_entry = train_path_entry[0]
path_entry

'/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/data_batch_1'

In [466]:
import random

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def preLoad(entry):
    data_dict = unpickle(entry)
    batch_label = data_dict[b'batch_label']
    data = data_dict[b'data']
    labels = data_dict[b'labels']
    filenames = data_dict[b'filenames']
    return data, labels

def preProcess(path_entry, num_classes = 10):
    data = []
    labels = []
    for entry in path_entry:
        temp_data, temp_labels = preLoad(entry)
        data.append(temp_data)
        labels.append(temp_labels)

    data = np.array(data)/255.0
    X =data.reshape(data.shape[0]*data.shape[1], 32, 32, 3)
    print("X.shape = ", X.shape)

    labels = np.array(labels)
    Y = labels.reshape(labels.shape[0]*labels.shape[1], 1)

    return X, Y

def getClassnames(names_entry):
    # Label: retrive classnames from the label
    with open(names_entry, 'rb') as file:
        names = pickle.load(file, encoding='bytes')
    label_names = names[b'label_names']
    return label_names

def showFig(ROWS, COLUMNS, data, labels, names_entry):
    # randomly sample
    indexList = random.sample([idx for idx in range(len(labels))], ROWS*COLUMNS)
    label_names = getClassnames(names_entry)
    
    # Image: show figures with their classnames on top
    fig, ax = plt.subplots(ROWS, COLUMNS, figsize=(COLUMNS*2,ROWS*2.5))
    for index in range(ROWS*COLUMNS):
        image = data[indexList[index], :]
        colorR, colorG, colorB = [image[:1024].reshape(32, 32), image[1024:2048].reshape(32, 32), image[2048:].reshape(32, 32)]
        image = np.zeros(shape = (32, 32, 3))
        image = np.dstack((colorR,colorG,colorB))
        ax[int(index/COLUMNS),  int(index%COLUMNS)].imshow(image)
        ax[int(index/COLUMNS),  int(index%COLUMNS)].set_title(label_names[labels[indexList[index]]].decode("utf-8"), fontsize = 20)
        ax[int(index/COLUMNS),  int(index%COLUMNS)].axis('off')
    
    
# names_entry = '/Users/yiyi/Google Drive/CS/Coursera/CIFAR-10/cifar-10-batches-py/batches.meta'
# classnames = getClassnames(names_entry)
# number_classes = len(classnames)
# print("number_classes = ", number_classes)
# showFig(10, 10, data, labels, names_entry)

# X, Y = preProcess(train_path_entry)
# Y = keras.utils.to_categorical(y, 10)
# showFig(10, 10, data, labels, names_entry)


X.shape =  (50000, 32, 32, 3)


In [ ]:
showFig(10, 10, data, labels, names_entry)

In [467]:
X, Y = preProcess(train_path_entry)
print("X.shape = ", X.shape)
print("Y.shape = ", Y.shape)

X.shape =  (50000, 32, 32, 3)
Y.shape =  (50000, 1)


In [470]:

def baseline_model():
    # creat model 
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding="same", input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size = (3, 3)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size = (3, 3)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(3072, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,  kernel_initializer = 'normal', activation = 'softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
cifar_model = baseline_model()
cifar_model.fit(X, Y, epochs = 10, batch_size = 8)

Epoch 1/10
50000/50000 [==============================] - 393s 8ms/step - loss: 1.5326 - acc: 0.4687
Epoch 2/10
41632/50000 [=======================>......] - ETA: 1:05 - loss: 1.2082 - acc: 0.5754